In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.stats import linregress 
import numpy as np
import math

In [ ]:
def GetNumberOfFiles(wavelength,folder_path):
    count = 0
    for filename in os.listdir(folder_path):
        if wavelength in filename:
            count += 1
    print(f"Number of files in the folder with {wavelength} in their name: {count}")
    return count


In [ ]:
def GetNumberOfFilesSens(wavelength,folder_path,sensorID):
    count = 0
    for filename in os.listdir(folder_path):
        if (sensorID in filename) and (wavelength in filename):
            count += 1
            print(f"ok {filename}")
    print(f"Number of files with {wavelength} and sensor name {sensorID}: {count}")
    return count


In [ ]:
def ProcessSensID(folder_path,wavelength,sensorID):   
    nfile = GetNumberOfFilesSens(wavelength,folder_path,sensorID)
    Tfull= pd.DataFrame()
    Tpedestalfull= pd.DataFrame()
    outputpath =  f'{folder_path}/plots'
    slopes = [] 
    intercepts= []
    rCoes = []
    stderr = []
    intercepts_stderr = []
    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
        print(f"Directory '{outputpath}' created.")
    else:
        print(f"Directory '{outputpath}' already exists.")

    if wavelength == '532':
        # Rename DataFrame columns
        strg_L = 'Laser Current (mA)'
        temp = '20'
    elif wavelength == '1064':
        strg_L = 'Laser Power (mW)'
        temp = '25'

    for i in range(1, nfile+1):
        filename = f'{folder_path}/test09052024_{sensorID}_{wavelength}_{i}.txt'
        T = pd.read_csv(filename,delimiter ='\t',header=None)
        #T.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']
        T.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts'] #'meanPM', 'stdPM' are not the correct ones now, we have 'TotalSum', 'TotalSquareSum' instead and we need to change that
        #T.columns = ['date', 'time', 'L', 'meanPM', 'stdPM', 'meanRefPD', 'stdRefPD', 'Temp', 'RH', 'samples'] #PD->power W;PM (V); sigmashould be sqrt('TotalSquareSum'/'TotalCounts')
        #<Date-Hour> <Laser Power  or Current> <Total Sum> <Total Square Sum> <PD Reference-Mean Voltage> <PD reference-Std> <Temperature> <RH%> <Total Counts>

        # Create a new DataFrame by copying the first row of the original DataFrame
        # we need to identify the pedestal in the data, i.e all the 0 in L
        

        #Tpedestal=T.head(1).copy()
        Tpedestal=T[(T['L']== 0)]
        Tpedestalid=T[(T['L']== 0)].index
        # Remove the rows from T that corresponds to the pedestal
        T = T.drop(Tpedestalid)
        
        # Now concatenate it to the previous pandas frame
        Tfull = pd.concat([Tfull, T])
        Tpedestalfull = pd.concat([Tpedestalfull, Tpedestal])
        # Create separate figures for each plot
        fig = plt.figure(i)
        plt.errorbar(T['L'], T['meanPM'], yerr=T['stdPM'], fmt='.', markersize=10, linewidth=1)
        plt.ylabel('Mean Optical Power (W)')
        plt.xlabel(strg_L)
        plt.grid()
        plt.title(f'Plot G {i}')
        plt.tight_layout()
        plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_PM_MM_{i}_L_PM.png',dpi=199)  # Display the current figure
        plt.close(fig)

        fig = plt.figure(i + 1)
        plt.errorbar(T['L'], T['meanRefPD'], yerr=T['stdRefPD'], fmt='.', markersize=10, linewidth=1)
        plt.ylabel('Mean ref PD (V)')
        plt.xlabel(strg_L)
        plt.grid()
        plt.title(f'Plot G {i}')
        plt.tight_layout()
        plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_PM_MM_{i}_L_PD.png',dpi=199)  # Display the current figure
        plt.close(fig)

        #Linear fit 
        res = linregress(T['meanRefPD'], T['meanPM'])
        slope = res.slope 
        intercept = res.intercept
        r=res.rvalue
        se = res.stderr
        intercept_stderr = res.intercept_stderr

        slopes.append(slope)
        intercepts.append(intercept)
        rCoes.append(r)
        stderr.append(se)
        intercepts_stderr.append(intercept_stderr)

        fig = plt.figure(i + 2)
        plt.errorbar(T['meanRefPD'], T['meanPM'], yerr=T['stdPM'], fmt='.', markersize=10, linewidth=1)
        plt.plot(T['meanRefPD'], intercept + slope*T['meanRefPD'], 'r', label='fitted line')
        plt.ylabel('Mean Optical Power (W)')
        plt.xlabel('Mean ref PD (V)')
        plt.grid()
        plt.title(f'Plot G {i}')
        plt.tight_layout()
        plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_PM_MM_{i}_PD_PM.png',dpi=199)  # Display the current figure
        plt.close(fig)

        plt.figure(101)
        plt.grid()
        plt.errorbar(T['L'], T['meanPM'], yerr=T['stdPM'], fmt='.', markersize=10, linewidth=1)
        plt.ylabel('Mean Optical Power (W)')
        plt.xlabel(strg_L)
        plt.title(f'all datasets')
        
        
        plt.figure(102)
        plt.grid()
        plt.errorbar(T['L'], T['meanRefPD'], yerr=T['stdRefPD'], fmt='.', markersize=10, linewidth=1)
        plt.ylabel('Mean ref PD (V)')
        plt.xlabel(strg_L)
        plt.title(f'all datasets')
        

        plt.figure(103)
        plt.grid()
        plt.errorbar(T['meanRefPD'], T['meanPM'], yerr=T['stdPM'], fmt='.', markersize=10, linewidth=1)
        plt.ylabel('Mean Optical Power (W)')
        plt.xlabel('Mean ref PD (V)')
        plt.title(f'all datasets')
    SignalAnalysisSensID(Tfull,nfile,slopes,stderr,intercepts,intercepts_stderr,rCoes,res,outputpath,wavelength,sensorID)
    PedestalAnalysis(Tpedestalfull['meanRefPD'],Tpedestalfull['stdRefPD'],'Optical Power (W)',outputpath, wavelength)
    PedestalAnalysis(Tpedestalfull['meanPM'],Tpedestalfull['stdPM'],'ref PD (V)',outputpath, wavelength)
    return Tfull, Tpedestalfull
 

In [ ]:
def SignalAnalysisSensID(Tfull,nfile,slopes,stderr,intercepts,intercepts_stderr,rCoes,res,outputpath,wavelength,sensorID):  
     
    x=Tfull['meanRefPD']
    y=Tfull['meanPM']

    res = linregress(x, y)
    datasets= nfile
    slopes_micro=[i * 1e6 for i in slopes]
    stderr_micro=[i * 1e6 for i in stderr]

    intercepts_micro=[i * 1e6 for i in intercepts]
    intercepts_stderr_micro=[i * 1e6 for i in intercepts_stderr]

    print(f'Calibration coefficient found as linear adjustment of all the dataset = {res.slope*1e6} +/- {res.stderr*1e6} in $\mu$W/V')

    print(f'Calibration coefficient found as mean of the linear adjustment of each dataset = {np.mean(slopes_micro)} +/- {np.std(slopes_micro)} in $\mu$W/V')
   
    #Errors in percentage 
    relativeError = (np.mean(slopes)-res.slope)/np.mean(slopes)*100
    dispersion = (np.max(slopes)-np.min(slopes))/np.max(slopes)*100
    print('relativeError = ',relativeError,'% and dispersion = ',dispersion,'%')
    
    # Customize the plot
    fig = plt.figure(100)
    plt.errorbar(range(len(slopes)), slopes_micro,stderr_micro, fmt='.', markersize=10, linewidth=1)
    plt.title(f'Calibration for Different Datasets at {wavelength} nm, relative error = {relativeError:.2e}, dispersion = {dispersion:.2f}')
    plt.xlabel('Dataset Index')
    plt.ylabel('Calibration coefficient ($\mu$W/V)')
    plt.xticks(range(datasets), [f'D {i+1}' for i in range(nfile)])  # Label the x-axis with dataset identifiers
    #plt.legend()
    plt.axhline(y=res.slope*1e6, color='red', linestyle='--', label='Horizontal Line at y=m_all')
    plt.axhline(y=np.mean(slopes_micro), color='purple', linestyle='--', label='Horizontal Line at y=m_av')
    plt.fill_between(range(-1,len(slopes)+1), (res.slope-res.stderr)*1e6, (res.slope+res.stderr)*1e6, color='red', alpha=0.3, label='Shaded Region')
    plt.fill_between(range(-1,len(slopes)+1), (np.mean(slopes_micro)-np.std(slopes_micro)), (np.mean(slopes_micro)+np.std(slopes_micro)), color='purple', alpha=0.2, label='Shaded Region')
    #plt.ylim([0.00064*1e6, 0.000665*1e6])
    plt.xlim([-1,nfile])
    plt.grid()
    # Show the plot
    #plt.show()
    plt.tight_layout()
    plt.savefig(f'{outputpath}/Laser_{wavelength}_Step_50_PM_MM_GAll_Cal_coeff.png',dpi=199)

    # Customize the plot
    fig = plt.figure(104)
    plt.errorbar(range(len(intercepts)), intercepts_micro,intercepts_stderr_micro, fmt='.', markersize=10, linewidth=1)
    plt.title(f'Offset for Different Datasets at {wavelength}')
    plt.xlabel('Dataset Index')
    plt.ylabel('Intercepts ($\mu$W)')
    plt.xticks(range(datasets), [f'D {i+1}' for i in range(nfile)])  # Label the x-axis with dataset identifiers
    #plt.legend()
    plt.axhline(y=res.intercept*1e6, color='red', linestyle='--', label='Horizontal Line at y=m_all')
    plt.axhline(y=np.mean(intercepts_micro), color='purple', linestyle='--', label='Horizontal Line at y=m_av')
    plt.fill_between(range(-1,len(intercepts)+1), (res.intercept-res.intercept_stderr)*1e6, (res.intercept+res.intercept_stderr)*1e6, color='red', alpha=0.3, label='Shaded Region')
    plt.fill_between(range(-1,len(intercepts)+1), (np.mean(intercepts_micro)-np.std(intercepts_micro)), (np.mean(intercepts_micro)+np.std(intercepts_micro)), color='purple', alpha=0.2, label='Shaded Region')
    #plt.ylim([0.00064*1e6, 0.000665*1e6])
    plt.xlim([-1,nfile])
    plt.grid()
    # Show the plot
    #plt.show()
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_PM_MM_All_Intercepts.png',dpi=199)

    plt.figure(101)
    plt.grid()
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_PM_MM_all_trend_LPower_PM.png',dpi=199)

    plt.figure(102)
    plt.grid()
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_PM_MM_all_trend_LPower_PD.png',dpi=199)

    plt.figure(103)
    plt.grid()
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_PM_MM_all_trend_PD_PM.png',dpi=199)

    return Tfull, slopes, intercepts, rCoes,stderr, res    

In [ ]:
def PedestalAnalysis(mean, std, name, outputpath, wavelength):
    samples = len(mean)
    fig = plt.figure(1103)
    # Mean and std deviation of the PM 
    plt.errorbar(range(samples), mean, yerr=std, fmt='.', markersize=10, linewidth=1)
    plt.axhline(y=np.mean(mean), color='purple', linestyle='--', label='Horizontal Line at y=m_av')
    plt.fill_between(range(samples), np.mean(mean)-np.std(mean), np.mean(mean)+np.std(mean), color='red', alpha=0.3, label='Shaded Region')

    plt.ylabel(f'Pedestal of {name}')
    plt.xlabel('Test number [#]')
    plt.grid()
    plt.tight_layout()
    plt.savefig(f'{outputpath}/Pedestal_{wavelength}_{name}.png',dpi=199)  # Display the current figure
    plt.close(fig)
    print(f'The pedestal of the values',name,' are: Mean=',np.mean(mean),' std=',np.mean(std))


In [ ]:
Tfull, Tpedestal =ProcessSensID('../data/','1064','0.0')

#Obtain a deep look of the data
#Tfull, Tpedestal = Process('../data/1064nm','1064')
#Tfull, Tpedestal = Process('../data/532nm','532')


In [9]:
folder_path='../data'
sensorsID={'0.0','0.1','0.2','0.3','1.0','1.1','1.2','1.3','2.0','2.1','2.2','2.3','3.0','3.1','3.2','3.3', '4.0','4.1','4.2','4.3'}
wavelength='1064'#'532' #1064
outputpath =  f'{folder_path}/plots'
i='1'
datasets= '1'



if wavelength == '532':
# Rename DataFrame columns
    strg_L = 'Laser Current (mA)'
    temp = '20'
elif wavelength == '1064':
    strg_L = 'Laser Power (mW)'
    temp = '25'

for l in sensorsID:
    sensorID=l

    filename = f'{folder_path}/test09052024_{sensorID}_{wavelength}_{i}.txt'
    T = pd.read_csv(filename,delimiter ='\t',header=None)
    T.columns = ['Date-Hour', 'L', 'TotalSum', 'TotalSquareSum', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']
    #T.columns = ['Date-Hour', 'L', 'meanPM', 'stdPM', 'meanRefPD', 'stdRefPD', 'Tem', 'RH', 'TotalCounts']

    #Tpedestal=T.head(1).copy()
    Tpedestal=T[(T['L']== 0)]
    #Tpedestalid=T[(T['L']== 0)].index
    # Remove the rows from T that corresponds to the pedestal
    T = T.drop(T[(T['L']== 0)].index)

    meanADC=T['TotalSum']/T['TotalCounts']
    stdADC=np.sqrt((T['TotalSquareSum']-T['TotalCounts']*meanADC**2)/(T['TotalCounts']-1))


    #Monica proposed from the baffle paper: 
    #Voltage/ADCcount =3.3V/2**10=0.00322 V/ADCcount
    #4.6 uW/ADCcount*1ADCcount/0.00322V = 1428 uW/V
    #Otger says 0.61mV/ADCcount

    meanPM=meanADC*0.61e-3 #0.61mV/ADCcount   #this conversion should be done in W not in V might check this one
    stdPM=stdADC*0.61e-3

    #meanPM=4.6e-6*meanADC/0.61e-3 #0.61mV/ADCcount   #this conversion might be incorrect
    #stdPM=stdADC*4.6e-6/0.61e-3


    T['meanADC']=meanADC
    T['stdADC']=stdADC

    T['meanPM']=meanPM
    T['stdPM']=stdPM

    res = linregress(T['meanRefPD'], T['meanADC'])
    slope = res.slope 
    intercept = res.intercept
    r=res.rvalue
    se = res.stderr
    intercept_stderr = res.intercept_stderr

    #this one is not correct
    resPM = linregress(T['meanRefPD'], T['meanPM'])
    slopePM = resPM.slope 
    interceptPM = resPM.intercept
    rPM=resPM.rvalue
    sePM = resPM.stderr
    intercept_stderrPM = resPM.intercept_stderr



    print(f'Calibration coefficient found as linear adjustment of all the dataset = {res.slope*1e6} +/- {res.stderr*1e6} in $muW/V')

    fig = plt.figure(1)
    plt.errorbar(T['L'], T['TotalSum'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
    plt.ylabel('ADC counts')
    plt.xlabel(strg_L)
    plt.grid()
    plt.title(f'Plot dataset {i}, sensorID {sensorID}, wavelength {wavelength} nm')
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_ADC_L_{datasets}.png')  # Display the current figure
    #plt.show()
    plt.close(fig)

    fig = plt.figure(2)
    plt.errorbar(T['L'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
    plt.ylabel('mean ADC counts')
    plt.xlabel(strg_L)
    plt.grid()
    plt.title(f'Plot dataset {i}, sensorID {sensorID}, wavelength {wavelength} nm')
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_meanADC_L_{datasets}.png',dpi=199)  # Display the current figure
    #plt.show()
    plt.close(fig)

    fig = plt.figure(3)
    plt.errorbar(T['L'], T['meanPM'], yerr=T['stdPM'], fmt='.', markersize=10, linewidth=1)
    plt.ylabel('Mean Optical Power (W) or (V) ?')
    plt.xlabel(strg_L)
    plt.grid()
    plt.title(f'Plot dataset {i}, sensorID {sensorID}, wavelength {wavelength} nm')
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_meanPM_L_{datasets}.png',dpi=199)  # Display the current figure
    #plt.show()
    plt.close(fig)

    fig = plt.figure(4)
    plt.errorbar(T['L'], T['meanRefPD'], yerr=T['stdRefPD'], fmt='.', markersize=10, linewidth=1)
    plt.ylabel('Mean ref PD (V)')
    plt.xlabel(strg_L)
    plt.grid()
    plt.title(f'Plot dataset {i}, sensorID {sensorID}, wavelength {wavelength} nm')
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_meanRefPD_L_{datasets}.png',dpi=199)  # Display the current figure
    #plt.show()
    plt.close(fig)



    #the label here are incorrect since meanPM=meanADC*0.61e-3 #0.61mV/ADCcoun. Monica proposed from the baffle paper: 
    #Voltage/ADCcount =3.3V/2**10=0.00322 V/ADCcount
    #4.6 uW/ADCcount*1ADCcount/0.00322V = 1428 uW/V

    fig = plt.figure(5)
    plt.errorbar(T['meanRefPD'], T['meanPM'], yerr=T['stdPM'], fmt='.', markersize=10, linewidth=1)
    plt.plot(T['meanRefPD'], interceptPM + slopePM*T['meanRefPD'], 'r', label='fitted line')
    plt.ylabel('Mean Optical Power (W) or (V) ?')
    plt.xlabel('Mean ref PD (V)')
    plt.grid()
    plt.title(f'Plot dataset {i}, sensorID {sensorID}, wavelength {wavelength} nm')
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_meanPM_meanRefPD_{datasets}.png',dpi=199)  # Display the current figure
    #plt.show()
    plt.close(fig)
    

    fig = plt.figure(6)
    plt.errorbar(T['meanRefPD'], T['meanADC'], yerr=T['stdADC'], fmt='.', markersize=10, linewidth=1)
    plt.plot(T['meanRefPD'], intercept + slope*T['meanRefPD'], 'r', label='fitted line')
    plt.ylabel('Mean Optical ADC counts')
    plt.xlabel('Mean ref PD (V)')
    plt.grid()
    plt.title(f'Plot dataset {i}, sensorID {sensorID}, wavelength {wavelength} nm')
    plt.tight_layout()
    plt.savefig(f'{outputpath}/test09052024_{sensorID}_{wavelength}_meanADC_meanRefPD_{datasets}.png',dpi=199)  # Display the current figure
    #plt.show()
    plt.close(fig)


Calibration coefficient found as linear adjustment of all the dataset = 442356852.965471 +/- 152511755.98299116 in $muW/V
Calibration coefficient found as linear adjustment of all the dataset = 5142421018.404169 +/- 243732701.58675945 in $muW/V
Calibration coefficient found as linear adjustment of all the dataset = 8619570848.977295 +/- 392544862.73272324 in $muW/V
Calibration coefficient found as linear adjustment of all the dataset = 6667214.800620846 +/- 736109.1449822279 in $muW/V
Calibration coefficient found as linear adjustment of all the dataset = 4684099779.219588 +/- 235961926.64983255 in $muW/V
Calibration coefficient found as linear adjustment of all the dataset = 8809715317.181376 +/- 400167857.69976604 in $muW/V
Calibration coefficient found as linear adjustment of all the dataset = 482918020.58024424 +/- 154328878.15763542 in $muW/V
Calibration coefficient found as linear adjustment of all the dataset = 388909992.42280924 +/- 149324868.91373855 in $muW/V
Calibration coef